In [49]:
import pandas as pd
from pipeline_executor import PipelineExecutor
import nlp_analysis.word_wizard as ww

topic = "'quantumcomputing'AND'research'"

pipeline_executor = PipelineExecutor()
quantum_df = pipeline_executor.execute(query=topic, max_articles=30, overwrite=True)

Getting news article info:  11%|█▏        | 10/87 [00:08<00:33,  2.32it/s]

In [16]:
wizard = ww.WordWizard(df=quantum_df)
wizard.create_word_embeddings(columns=['body'], lean=True) \
    .cluster_embeddings('body')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating embeddings for column(s): ['body']: 100%|██████████| 1/1 [02:42<00:00, 162.30s/it]


Optimal K: 2


/Users/florian/Documents/Learning/MiBA/classes/term3/Capstone/MIBA-2023-CAPSTONE-RB-NLP/nlp_analysis/word_wizard.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = kmeans.labels_
/Users/florian/Documents/Learning/MiBA/classes/term3/Capstone/MIBA-2023-CAPSTONE-RB-NLP/nlp_analysis/word_wizard.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column + self.MEDOID_SUFFIX] = False


In [17]:
# checking correctness of medoids (3 per cluster)
wizard.df.groupby('body_word_embeddings_clusters').agg({'body_word_embeddings_clusters_medoids': "sum"})

,body_word_embeddings_clusters_medoids
body_word_embeddings_clusters,
0.0,3
1.0,3


In [48]:
# ensuring that medioids have different values
len(wizard.df[wizard.df['body_word_embeddings_clusters_medoids'] == 1]['body_word_embeddings'].drop_duplicates()) == len(wizard.df[wizard.df['body_word_embeddings_clusters_medoids'] == 1])

True

# Summarization

In [20]:
import transformers
from transformers import BertTokenizer
summarizer = transformers.pipeline("summarization", model="facebook/bart-large-cnn")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [43]:
sub_df = wizard.df[wizard.df['body_word_embeddings_clusters_medoids'] == 1]
for i in range(len(sub_df)):
    print(len(tokenizer.tokenize(sub_df.iloc[i]['body'])))
    try:
        summary = summarizer(sub_df.iloc[i]['body'])
    except:
        summary = 'toolong'
    print('cluster = ', 
          sub_df.iloc[i]['body_word_embeddings_clusters'],
          'summary', summary)

1254
cluster =  0.0 summary toolong
1362
cluster =  1.0 summary toolong
417
cluster =  0.0 summary [{'summary_text': 'Google is expanding its quantum computing research investments with Macquarie University and the University of Technology Sydney (UTS) The search giant is launching new partnership with University of Sydney and UNSW. Google’s Digital Future Initiative was announced in November last year, and earlier this month, Google appointed Grace Chung and Peter Bartlett to jointly lead the project.'}]
1584
cluster =  1.0 summary toolong
634
cluster =  1.0 summary [{'summary_text': 'IBM announced a new approach that it has developed in partnership with UC Berkley. The approach effectively counteracts the noise in IBM’s Eagle quantum computer. This is just one significant step toward quantum supremacy – a much-awaited threshold where the ability of quantum computers demonstrably exceeds that of conventional ones.'}]
2587
cluster =  0.0 summary toolong
